###### General Steps to Follow
1. Importing Packages
1. Criteria 1 - Poses
1. Criteria 2 - Poses
1. Criteria 3 - Poses
1. Criteria 4 - Poses
1. Full Test

# -------------------------------------------------------------------------------------------------------------

## 1) Importing Packages

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, GRU
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import tensorflow as tf

2024-04-27 21:03:47.178155: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 21:03:48.007245: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# -------------------------------------------------------------------------------------------------------------

In [2]:
def evaluate(output, y_true):
    y_pred = np.zeros((output.shape[0],1), dtype = "int")
    y_pred[(output > 0.5)] = 1
    y_pred = y_pred.reshape(y_pred.shape[0])
    print("accuracy =", accuracy_score(y_true, y_pred))

In [9]:
def reshape_input_sequence(data, shape):
    new_data = []
    for training_example in data:
        new_data.append(training_example[0:shape])
    new_data = np.array(new_data)
    return new_data

## 2) Criteria 1 - Poses

#### Loading training data

In [3]:
criteria1_data_poses = np.load("../data/processed/bicep/poses/training_data_bicep_poses_criteria_1.npy", allow_pickle = True)

In [4]:
len(criteria1_data_poses)

954

#### Getting X_train, X_test, y_train, y_test

In [11]:
X = np.vstack(criteria1_data_poses[:, 0])
y = np.vstack(criteria1_data_poses[:, 1])
X_train_criteria1_poses, X_test_criteria1_poses, y_train_criteria1_poses, y_test_criteria1_poses = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [12]:
print("X_train_criteria1_poses shape:", X_train_criteria1_poses.shape)
print("y_train_criteria1_poses shape:", y_train_criteria1_poses.shape)
print("X_test_criteria1_poses shape:", X_test_criteria1_poses.shape)
print("y_test_criteria1_poses shape:", y_test_criteria1_poses.shape)

X_train_criteria1_poses shape: (763, 600, 24)
y_train_criteria1_poses shape: (763, 1)
X_test_criteria1_poses shape: (191, 600, 24)
y_test_criteria1_poses shape: (191, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [13]:
new_length = 150
X_train_criteria1_poses = reshape_input_sequence(X_train_criteria1_poses, new_length)
X_test_criteria1_poses = reshape_input_sequence(X_test_criteria1_poses, new_length)
print("X_train_criteria1_poses shape:", X_train_criteria1_poses.shape)
print("y_train_criteria1_poses shape:", y_train_criteria1_poses.shape)
print("X_test_criteria1_poses shape:", X_test_criteria1_poses.shape)
print("y_test_criteria1_poses shape:", y_test_criteria1_poses.shape)

X_train_criteria1_poses shape: (763, 150, 24)
y_train_criteria1_poses shape: (763, 1)
X_test_criteria1_poses shape: (191, 150, 24)
y_test_criteria1_poses shape: (191, 1)


#### Building and training the model

In [223]:
criteria1_poses_model = Sequential(
    [
        GRU(units=256, input_shape=(X_train_criteria1_poses.shape[1], X_train_criteria1_poses.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 64, return_sequences=True),
        BatchNormalization(),
        GRU(units = 32),
        BatchNormalization(),
        #Dense(256, activation = "relu"),
        #BatchNormalization(),
        #Dense(128, activation = "relu"),
        #BatchNormalization(),
        #Dense(64, activation = "relu"),
         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], 
    name="criteria1_poses_model"
)

criteria1_poses_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria1_poses_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_94 (GRU)                    │ (None, 150, 256)       │       216,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_96          │ (None, 150, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_95 (GRU)                    │ (None, 150, 128)       │       148,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_97          │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_96 (GRU)                    │ (None, 150, 64)        │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_98          │ (None, 150, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_97 (GRU)                    │ (None, 32)             │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_99          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 413,409 (1.58 MB)

 Trainable params: 412,449 (1.57 MB)

 Non-trainable params: 960 (3.75 KB)

In [259]:
criteria1_poses_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.0001),
                              loss = BinaryCrossentropy(from_logits = True))

In [267]:
criteria1_poses_model.fit(X_train_criteria1_poses, y_train_criteria1_poses, epochs = 10, batch_size = 32)

#### Saving the model

In [264]:
criteria1_poses_model.save("../models/other/bicep/poses/bicep_criteria1_poses_model_v4.h5")

#### Loading the model

In [256]:
criteria1_poses_model = tf.keras.models.load_model("../models/other/bicep/poses/bicep_criteria1_poses_model_v2.h5")

#### Evaluating the model

###### Train Data

In [262]:
y_pred = criteria1_poses_model.predict(X_train_criteria1_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria1_poses)

24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 118ms/step
accuracy = 0.9882044560943644


###### Test Data

In [261]:
y_pred = criteria1_poses_model.predict(X_test_criteria1_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria1_poses)

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step
accuracy = 0.9633507853403142


# -------------------------------------------------------------------------------------------------------------

## 3) Criteria 2 - Poses

#### Loading training data

In [265]:
criteria2_data_poses = np.load("../data/processed/bicep/poses/training_data_bicep_poses_criteria_2.npy", allow_pickle = True)

In [266]:
len(criteria2_data_poses)

1064

#### Getting X_train, X_test, y_train, y_test

In [268]:
X = np.vstack(criteria2_data_poses[:, 0])
y = np.vstack(criteria2_data_poses[:, 1])
X_train_criteria2_poses, X_test_criteria2_poses, y_train_criteria2_poses, y_test_criteria2_poses = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [269]:
print("X_train_criteria2_poses shape:", X_train_criteria2_poses.shape)
print("y_train_criteria2_poses shape:", y_train_criteria2_poses.shape)
print("X_test_criteria2_poses shape:", X_test_criteria2_poses.shape)
print("y_test_criteria2_poses shape:", y_test_criteria2_poses.shape)

X_train_criteria2_poses shape: (851, 600, 24)
y_train_criteria2_poses shape: (851, 1)
X_test_criteria2_poses shape: (213, 600, 24)
y_test_criteria2_poses shape: (213, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [270]:
new_length = 150
X_train_criteria2_poses = reshape_input_sequence(X_train_criteria2_poses, new_length)
X_test_criteria2_poses = reshape_input_sequence(X_test_criteria2_poses, new_length)
print("X_train_criteria2_poses shape:", X_train_criteria2_poses.shape)
print("y_train_criteria2_poses shape:", y_train_criteria2_poses.shape)
print("X_test_criteria2_poses shape:", X_test_criteria2_poses.shape)
print("y_test_criteria2_poses shape:", y_test_criteria2_poses.shape)

X_train_criteria2_poses shape: (851, 150, 24)
y_train_criteria2_poses shape: (851, 1)
X_test_criteria2_poses shape: (213, 150, 24)
y_test_criteria2_poses shape: (213, 1)


#### Building and training the model

In [271]:
criteria2_poses_model = Sequential(
    [
        GRU(units=200, input_shape=(X_train_criteria2_poses.shape[1], X_train_criteria2_poses.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 256, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 128, return_sequences=True),
        BatchNormalization(),
        GRU(units = 32),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], name = "criteria2_poses_model"
)
criteria2_poses_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria2_poses_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_98 (GRU)                    │ (None, 150, 200)       │       135,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_100         │ (None, 150, 200)       │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_99 (GRU)                    │ (None, 150, 128)       │       126,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_101         │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_100 (GRU)                   │ (None, 150, 128)       │        99,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_102         │ (None, 150, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_101 (GRU)                   │ (None, 32)             │        15,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_103         │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 378,929 (1.45 MB)

 Trainable params: 377,953 (1.44 MB)

 Non-trainable params: 976 (3.81 KB)

In [276]:
criteria2_poses_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.001),
                              loss = BinaryCrossentropy(from_logits = True))

In [328]:
criteria2_poses_model.fit(X_train_criteria2_poses, y_train_criteria2_poses, epochs = 10, batch_size = 32)

#### Saving the model

In [281]:
criteria2_poses_model.save("../models/other/bicep/poses/bicep_criteria2_poses_model_v1.h5")

#### Loading the model

In [283]:
criteria2_poses_model = tf.keras.models.load_model("../models/other/bicep/poses/bicep_criteria2_poses_model_v1.h5")

#### Evaluating the model

###### Train Data

In [284]:
y_pred = criteria2_poses_model.predict(X_train_criteria2_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria2_poses)

27/27 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step
accuracy = 0.9494712103407755


###### Test Data

In [285]:
y_pred = criteria2_poses_model.predict(X_test_criteria2_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria2_poses)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 123ms/step
accuracy = 0.9530516431924883


# -------------------------------------------------------------------------------------------------------------

## 4) Criteria 3 - Poses

#### Loading training data

In [286]:
criteria3_data_poses = np.load("../data/processed/bicep/poses/training_data_bicep_poses_criteria_3.npy", allow_pickle = True)

In [287]:
len(criteria3_data_poses)

767

#### Getting X_train, X_test, y_train, y_test

In [288]:
X = np.vstack(criteria3_data_poses[:, 0])
y = np.vstack(criteria3_data_poses[:, 1])
X_train_criteria3_poses, X_test_criteria3_poses, y_train_criteria3_poses, y_test_criteria3_poses = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [289]:
print("X_train_criteria3_poses shape:", X_train_criteria3_poses.shape)
print("y_train_criteria3_poses shape:", y_train_criteria3_poses.shape)
print("X_test_criteria3_poses shape:", X_test_criteria3_poses.shape)
print("y_test_criteria3_poses shape:", y_test_criteria3_poses.shape)

X_train_criteria3_poses shape: (613, 600, 24)
y_train_criteria3_poses shape: (613, 1)
X_test_criteria3_poses shape: (154, 600, 24)
y_test_criteria3_poses shape: (154, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [290]:
new_length = 150
X_train_criteria3_poses = reshape_input_sequence(X_train_criteria3_poses, new_length)
X_test_criteria3_poses = reshape_input_sequence(X_test_criteria3_poses, new_length)
print("X_train_criteria3_poses shape:", X_train_criteria3_poses.shape)
print("y_train_criteria3_poses shape:", y_train_criteria3_poses.shape)
print("X_test_criteria3_poses shape:", X_test_criteria3_poses.shape)
print("y_test_criteria3_poses shape:", y_test_criteria3_poses.shape)

X_train_criteria3_poses shape: (613, 150, 24)
y_train_criteria3_poses shape: (613, 1)
X_test_criteria3_poses shape: (154, 150, 24)
y_test_criteria3_poses shape: (154, 1)


#### Building and training the model

In [339]:
criteria3_poses_model = Sequential(
    [
        GRU(units=256, input_shape=(X_train_criteria3_poses.shape[1], X_train_criteria3_poses.shape[2]), return_sequences=True),
        #LSTM(units = 1024, return_sequences=True),
        #BatchNormalization(),
        #LSTM(units = 512, return_sequences=True),
        BatchNormalization(),
        GRU(units = 190, return_sequences=True),
        BatchNormalization(),
        #GRU(units = 150, return_sequences=True),
        #BatchNormalization(),
        GRU(units = 64, return_sequences=True),
        BatchNormalization(),
        GRU(units = 20),
        BatchNormalization(),
#         Dense(256, activation = "relu"),
#         #BatchNormalization(),
#         Dense(128, activation = "relu"),
#         #BatchNormalization(),
#         Dense(64, activation = "relu"),
#         #BatchNormalization(),
        Dense(1, activation = "linear")
    ], name = "criteria3_poses_model"
)
criteria3_poses_model.summary()

/home/amro/anaconda3/envs/ai1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "criteria3_poses_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru_123 (GRU)                   │ (None, 150, 256)       │       216,576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_125         │ (None, 150, 256)       │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_124 (GRU)                   │ (None, 150, 190)       │       255,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_126         │ (None, 150, 190)       │           760 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_125 (GRU)                   │ (None, 150, 64)        │        49,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_127         │ (None, 150, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_126 (GRU)                   │ (None, 20)             │         5,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_128         │ (None, 20)             │            80 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 528,389 (2.02 MB)

 Trainable params: 527,329 (2.01 MB)

 Non-trainable params: 1,060 (4.14 KB)

In [376]:
criteria3_poses_model.compile(metrics = ['accuracy'], 
                               optimizer = Adam(learning_rate = 0.0000000001),
                              loss = BinaryCrossentropy(from_logits = True))

In [1]:
criteria3_poses_model.fit(X_train_criteria3_poses, y_train_criteria3_poses, epochs = 10, batch_size = 32)

#### Saving the model

In [375]:
criteria3_poses_model.save("../models/other/bicep/poses/bicep_criteria3_poses_model_v2.h5")

#### Loading the model

In [ ]:
criteria3_poses_model = load_model("../models/other/bicep/poses/bicep_criteria3_poses_model_v1.h5")

#### Evaluating the model

###### Train Data

In [378]:
y_pred = criteria3_poses_model.predict(X_train_criteria3_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_train_criteria3_poses)

20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 142ms/step
accuracy = 0.9314845024469821


###### Test Data

In [379]:
y_pred = criteria3_poses_model.predict(X_test_criteria3_poses)
y_pred = tf.nn.sigmoid(y_pred)
evaluate(y_pred, y_test_criteria3_poses)

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 117ms/step
accuracy = 0.8961038961038961


# -------------------------------------------------------------------------------------------------------------

## 5) Criteria 4 - Poses

#### Loading training data

In [382]:
criteria4_data_poses = np.load("../data/processed/bicep/poses/training_data_bicep_poses_criteria_4.npy", allow_pickle = True)

In [383]:
len(criteria4_data_poses)

803

#### Getting X_train, X_test, y_train, y_test

In [384]:
X = np.vstack(criteria4_data_poses[:, 0])
y = np.vstack(criteria4_data_poses[:, 1])
X_train_criteria4_poses, X_test_criteria4_poses, y_train_criteria4_poses, y_test_criteria4_poses = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [385]:
print("X_train_criteria4_poses shape:", X_train_criteria4_poses.shape)
print("y_train_criteria4_poses shape:", y_train_criteria4_poses.shape)
print("X_test_criteria4_poses shape:", X_test_criteria4_poses.shape)
print("y_test_criteria4_poses shape:", y_test_criteria4_poses.shape)

X_train_criteria4_poses shape: (642, 600, 24)
y_train_criteria4_poses shape: (642, 1)
X_test_criteria4_poses shape: (161, 600, 24)
y_test_criteria4_poses shape: (161, 1)


#### Reducing the length of the input sequence in X_train, X_test

In [386]:
new_length = 150
X_train_criteria4_poses = reshape_input_sequence(X_train_criteria4_poses, new_length)
X_test_criteria4_poses = reshape_input_sequence(X_test_criteria4_poses, new_length)
print("X_train_criteria4_poses shape:", X_train_criteria4_poses.shape)
print("y_train_criteria4_poses shape:", y_train_criteria4_poses.shape)
print("X_test_criteria4_poses shape:", X_test_criteria4_poses.shape)
print("y_test_criteria4_poses shape:", y_test_criteria4_poses.shape)

X_train_criteria4_poses shape: (642, 150, 24)
y_train_criteria4_poses shape: (642, 1)
X_test_criteria4_poses shape: (161, 150, 24)
y_test_criteria4_poses shape: (161, 1)


#### Building and training the model

In [50]:
criteria4_poses_model = Sequential(
    [
        LSTM(units=50, input_shape=(X_train_criteria4_poses.shape[1], X_train_criteria4_poses.shape[2])),
        Dense(1, activation = "sigmoid")
    ], name = "criteria4_poses_model"
)
criteria4_poses_model.summary()

Model: "criteria4_poses_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, 50)                15000     
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 15051 (58.79 KB)
Trainable params: 15051 (58.79 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
criteria4_poses_model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [52]:
criteria4_poses_model.fit(X_train_criteria4_poses,y_train_criteria4_poses, epochs = 10)

Epoch 1/10
21/21 [==============================] - 5s 187ms/step - loss: 0.6757 - accuracy: 0.6262
Epoch 2/10
21/21 [==============================] - 4s 176ms/step - loss: 0.6660 - accuracy: 0.6262
Epoch 3/10
21/21 [==============================] - 4s 187ms/step - loss: 0.6621 - accuracy: 0.6262
Epoch 4/10
21/21 [==============================] - 4s 198ms/step - loss: 0.6622 - accuracy: 0.6262
Epoch 5/10
21/21 [==============================] - 4s 185ms/step - loss: 0.6621 - accuracy: 0.6262
Epoch 6/10
21/21 [==============================] - 4s 188ms/step - loss: 0.6619 - accuracy: 0.6262
Epoch 7/10
21/21 [==============================] - 4s 184ms/step - loss: 0.6655 - accuracy: 0.6262
Epoch 8/10
21/21 [==============================] - 4s 187ms/step - loss: 0.6626 - accuracy: 0.6262
Epoch 9/10
21/21 [==============================] - 4s 167ms/step - loss: 0.6621 - accuracy: 0.6262
Epoch 10/10
21/21 [==============================] - 4s 179ms/step - loss: 0.6617 - accuracy: 0.6262

#### Saving the model

In [ ]:
criteria4_poses_model.save("model.h5")

#### Loading the model

In [ ]:
loaded_model = load_model('my_model.h5')

#### Evaluating the model

###### Train Data

In [53]:
y_pred = criteria4_poses_model.predict(X_train_criteria4_poses)
evaluate(y_pred, y_train_criteria4_poses)

21/21 [==============================] - 1s 57ms/step
accuracy = 0.6261682242990654


###### Test Data

In [54]:
y_pred = criteria4_poses_model.predict(X_test_criteria4_poses)
evaluate(y_pred, y_test_criteria4_poses)

6/6 [==============================] - 0s 40ms/step
accuracy = 0.577639751552795


# -------------------------------------------------------------------------------------------------------------

## 6) Full Test

In [396]:
def test_models(input_sample):
    input_sample = input_sample.reshape((1, input_sample.shape[0], input_sample.shape[1]))
    y_pred1 = criteria1_poses_model.predict(input_sample)
    y_pred1 = tf.nn.sigmoid(y_pred1)
    y_pred2 = criteria2_poses_model.predict(input_sample)
    y_pred2 = tf.nn.sigmoid(y_pred2)
    y_pred3 = criteria3_poses_model.predict(input_sample)
    y_pred3 = tf.nn.sigmoid(y_pred3)
    print("Criteria1:", y_pred1[0][0].numpy())
    print("Criteria2:", y_pred2[0][0].numpy())
    print("Criteria3:", y_pred3[0][0].numpy())

In [397]:
import random

In [410]:
num = random.randint(0,X_test_criteria4_poses.shape[0]-1)
input_sample = X_test_criteria4_poses[num]
print(y_test_criteria4_poses[num])
test_models(input_sample)

[0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Criteria1: 0.9503206
Criteria2: 0.63359296
Criteria3: 0.0653212


In [387]:
#X_test_criteria4_poses[0][0:10]

In [388]:
#X_test_criteria4_poses[1][0:10]